In [ ]:
## generate departure time based on frequency 
# (for the lines we know frequeny, e.g., bus routes in Kyoto city)

import pandas as pd
from datetime import datetime, timedelta

# Load the Excel file
file_path = r"C:\Users\zhous\Desktop\zhouyh\berlin project\bus\bus_kyoto.xlsx"
df = pd.read_excel(file_path, sheet_name="frequency")

# Get the hours from the second row (starting from column 7, index 6)
hours = df.iloc[0, 6:].tolist()  # Extract the hours from the second row (index 1, starting from column 7)

# Function to generate departure times for a given number of runs
def generate_departure_times(hour, num_runs):
    # Convert hour to integer if it's not already
    hour = int(hour)
    
    # Format hour as two digits and convert to time
    start_time = datetime.strptime(f"{hour:02d}:00", "%H:%M")  
    
    if num_runs > 1:
        interval = timedelta(hours=1) / num_runs  # Calculate the interval between runs
    else:
        interval = timedelta(hours=1)  # If only one run, it's at the start of the hour

    times = []
    for run in range(num_runs):
        departure_time = start_time + run * interval
        times.append(departure_time.strftime("%H:%M:%S"))
    return times

# Initialize an empty list to store each row (each bus line's departure times)
bus_line_departures = []

# Iterate over each row (bus line) and generate departure times for each hour
for index, row in df.iterrows():
    if index >= 1:  # Skip the first two rows (explanation and hours row)
        # Concatenate columns 1, 2, and 3 to create the bus line name
        bus_line = f"{row[1]}_{row[2]}_{row[3]}"  # Assuming bus line name is in columns 1, 2, and 3 
        
        # Create an empty list to store all departure times for this bus line
        all_departure_times = []
        
        # Start from column 7 and iterate over the hours
        for hour, col in zip(hours, range(6, len(df.columns))):  # Adjust for column indexing starting from 7 (index 6)
            num_runs = row[col]  # Get the number of runs for this hour
            
            if pd.notna(num_runs) and num_runs > 0:  # Only process if there are bus runs
                times = generate_departure_times(hour, num_runs)
                all_departure_times.extend(times)  # Append the generated times to the list
        
        # Append the bus line name and its combined departure times as a single list
        bus_line_departures.append([bus_line, all_departure_times])

# Convert the departure times list into a DataFrame
result_df = pd.DataFrame(bus_line_departures, columns=['Bus Line', 'Departure Times'])

# Save the departure times to a new Excel file
result_df.to_excel("bus_departuretimes.xlsx", index=False)

# Print result to check
print(result_df)

In [ ]:
## generate time offset based on distance of shortest path
import pandas as pd
from datetime import timedelta

# Load the CSV file
file_path = r"C:\Users\zhous\Desktop\zhouyh\berlin project\bus\bus sequence_example.csv"  # Change this to your file path
df = pd.read_csv(file_path)

# Convert speed to meters per second (assume 25 km/h = 25 * 1000 / 3600 m/s)
speed_mps = 25 * 1000 / 3600

# Ensure the "Distance" column is numeric
df["Distance"] = pd.to_numeric(df["Distance"], errors="coerce")  # Handle non-numeric values safely

# Calculate travel time (seconds) from the distance column
df["travel_time"] = df["Distance"] / speed_mps

# Compute cumulative sum of travel time **relative to the first stop of each line**
df["arrivalOffset"] = df.groupby(["Bus operater", "Bus route"])["travel_time"].cumsum()
df["departureOffset"] = df["arrivalOffset"]  # Assuming no waiting time

# Ensure the first stop of each route has arrivalOffset and departureOffset as "00:00:00"
df.loc[df["Sequence"] == 1, ["arrivalOffset", "departureOffset"]] = 0  # the first stop of each line

# Convert seconds to HH:MM:SS format
def convert_to_hms(seconds):
    if seconds == 0:
        return "00:00:00"
    return str(timedelta(seconds=int(seconds)))

df["arrivalOffset"] = df["arrivalOffset"].apply(convert_to_hms)
df["departureOffset"] = df["departureOffset"].apply(convert_to_hms)

# Save the updated file
df.to_excel(r"C:\Users\zhous\Desktop\zhouyh\berlin project\bus\updated_bus_schedule.xlsx", index=False)

print("Offsets calculated and saved to updated_bus_schedule.xlsx")

In [ ]:
## generate xml from bus route sequence table
import pandas as pd
import xml.etree.ElementTree as ET
from datetime import datetime, timedelta

# Load the CSV data
df = pd.read_excel(r"C:\Users\zhous\Desktop\zhouyh\berlin project\bus\updated_bus_schedule.xlsx")

# Create the root <transitSchedule> element
transit_schedule = ET.Element("transitSchedule")

# Create the <transitStops> section
transit_stops = ET.SubElement(transit_schedule, "transitStops")

# Create a dictionary to store station ids and their coordinates
station_ids = {}
for idx, row in df.iterrows():
    # Use a combination of Bus route and Sequence as station_id
    station_id = f"{row['Bus route']}_{row['Sequence']}"
    if station_id not in station_ids:
        station_ids[station_id] = {
            "x": row['latitude'],
            "y": row['longitude'],
            "name": row['Bus station']
        }

# Create <stopFacility> for each station
for station_id, data in station_ids.items():
    stop_facility = ET.SubElement(
        transit_stops, "stopFacility", 
        id=station_id, 
        x=str(data["x"]), 
        y=str(data["y"]), 
        name=data["name"], 
        linkRefId=f"pt_{station_id}",
        isBlocking="false"
    )

# Function to generate times every 15 minutes from 6 AM to 9 PM
def generate_departure_times(start_time="06:00:00", end_time="21:00:00", interval_minutes=30):
    # Convert string times to datetime objects
    start = datetime.strptime(start_time, "%H:%M:%S")
    end = datetime.strptime(end_time, "%H:%M:%S")
    times = []
    
    # Generate times in the interval
    while start <= end:
        times.append(start.strftime("%H:%M:%S"))
        start += timedelta(minutes=interval_minutes)
    
    return times

# Generate departure times every 15 minutes from 6 AM to 9 PM
departure_times = generate_departure_times()

# Create the <transitLine> section for each unique Bus operator and Bus route
for (operator, route) in df[['Bus operator', 'Bus route']].drop_duplicates().values:
    transit_line = ET.SubElement(transit_schedule, "transitLine", id=f"{operator}_{route}")
    
    # Create the <transitRoute> section
    transit_route = ET.SubElement(transit_line, "transitRoute", id=f"{operator}_{route}_1")
    transport_mode = ET.SubElement(transit_route, "transportMode")
    transport_mode.text = "bus"
    
    # Create the <routeProfile> section
    route_profile = ET.SubElement(transit_route, "routeProfile")
    route_data = df[(df['Bus operator'] == operator) & (df['Bus route'] == route)].sort_values('Sequence')  # Sort by sequence for the route
    for _, row in route_data.iterrows():
        stop_ref = ET.SubElement(
            route_profile, "stop", 
            refId=f"{row['Bus route']}_{row['Sequence']}", 
            arrivalOffset=row['arrivalOffset'], 
            departureOffset=row['departureOffset'], 
            awaitDeparture="true"
        )
    
    # Create the <route> section (links between stations)
    route_element = ET.SubElement(transit_route, 'route')
    stops = route_data['Bus route'].astype(str) + '_' + route_data['Sequence'].astype(str)  # Create the stop IDs
    for stop_id in stops:
        ET.SubElement(route_element, 'link', {'refId': f"pt_{stop_id}"})  # Add the link with refId based on stop_id

    # Create the <departures> section
    departures = ET.SubElement(transit_route, "departures")
    for idx, departure_time in enumerate(departure_times):
        # Create a departure for each departure time, with a unique ID and vehicleRefId
        departure = ET.SubElement(
            departures, "departure", 
            id=str(idx + 1),  # Unique ID for each departure
            departureTime=departure_time, 
            vehicleRefId=f"pt_{route}_vehicle_{idx + 1}"
        )

# Write the XML tree to a file
tree = ET.ElementTree(transit_schedule)
tree.write(r"C:\Users\zhous\Desktop\zhouyh\berlin project\bus\bus_schedule_1.xml")

In [ ]:
## generate xml from bus route sequence table 
# and add reverse route
import pandas as pd
import xml.etree.ElementTree as ET
from datetime import datetime, timedelta

# Load the CSV data
df = pd.read_excel(r"C:\Users\zhous\Desktop\zhouyh\berlin project\bus\updated_bus_schedule.xlsx")

# Create the root <transitSchedule> element
transit_schedule = ET.Element("transitSchedule")

# Create the <transitStops> section
transit_stops = ET.SubElement(transit_schedule, "transitStops")

# Create a dictionary to store station IDs and their coordinates
station_ids = {}
for _, row in df.iterrows():
    station_id = f"{row['Bus route']}_{row['Sequence']}"
    if station_id not in station_ids:
        station_ids[station_id] = {
            "x": row['latitude'],
            "y": row['longitude'],
            "name": row['Bus station']
        }

# Create <stopFacility> for each station
for station_id, data in station_ids.items():
    stop_facility = ET.SubElement(
        transit_stops, "stopFacility", 
        id=station_id, 
        x=str(data["x"]), 
        y=str(data["y"]), 
        name=data["name"], 
        linkRefId=f"pt_{station_id}",
        isBlocking="false"
    )

# Function to generate times every 30 minutes from 6 AM to 9 PM
def generate_departure_times(start_time="06:00:00", end_time="21:00:00", interval_minutes=30):
    start = datetime.strptime(start_time, "%H:%M:%S")
    end = datetime.strptime(end_time, "%H:%M:%S")
    times = []
    
    while start <= end:
        times.append(start.strftime("%H:%M:%S"))
        start += timedelta(minutes=interval_minutes)
    
    return times

# Generate departure times every 30 minutes
departure_times = generate_departure_times()

# Create the <transitLine> section for each unique Bus operator and Bus route
for (operator, route) in df[['Bus operator', 'Bus route']].drop_duplicates().values:
    transit_line = ET.SubElement(transit_schedule, "transitLine", id=f"{operator}_{route}")

    # Process forward and reverse routes
    for direction in [1, 2]:
        route_suffix = "_forward" if direction == 1 else "_reverse"
        transit_route = ET.SubElement(transit_line, "transitRoute", id=f"{operator}_{route}{route_suffix}")
        transport_mode = ET.SubElement(transit_route, "transportMode")
        transport_mode.text = "bus"

        # Get data for this route and reverse it for the second loop
        route_data = df[(df['Bus operator'] == operator) & (df['Bus route'] == route)].sort_values('Sequence')
        
        if direction == 2:  # Reverse route processing
            route_data = route_data.iloc[::-1].reset_index(drop=True)

            # Convert time offsets to timedelta
            route_data['departureOffset'] = pd.to_timedelta(route_data['departureOffset'])
            route_data['arrivalOffset'] = pd.to_timedelta(route_data['arrivalOffset'])

            # Compute time differences for reverse route
            new_departure_offsets = [timedelta(seconds=0)]  # First stop is 00:00:00
            for i in range(1, len(route_data)):
                time_diff = abs(route_data['departureOffset'].iloc[i] - route_data['departureOffset'].iloc[i - 1])
                new_departure_offsets.append(new_departure_offsets[i - 1] + time_diff)

            # Assign new offsets
            route_data['departureOffset'] = new_departure_offsets
            route_data['arrivalOffset'] = new_departure_offsets

            # Convert timedelta to hh:mm:ss format (without "0 days")
            def timedelta_to_str(td):
                # Extract hours, minutes, seconds and format them
                total_seconds = int(td.total_seconds())
                hours = total_seconds // 3600
                minutes = (total_seconds % 3600) // 60
                seconds = total_seconds % 60
                return f"{hours:02}:{minutes:02}:{seconds:02}"

            # Apply formatting to the offsets
            route_data['departureOffset'] = route_data['departureOffset'].apply(timedelta_to_str)
            route_data['arrivalOffset'] = route_data['arrivalOffset'].apply(timedelta_to_str)

        # Create the <routeProfile> section
        route_profile = ET.SubElement(transit_route, "routeProfile")
        for _, row in route_data.iterrows():
            ET.SubElement(
                route_profile, "stop", 
                refId=f"{row['Bus route']}_{row['Sequence']}", 
                arrivalOffset=row['arrivalOffset'], 
                departureOffset=row['departureOffset'], 
                awaitDeparture="true"
            )

        # Create the <route> section (links between stations)
        route_element = ET.SubElement(transit_route, 'route')
        stops = route_data['Bus route'].astype(str) + '_' + route_data['Sequence'].astype(str)
        for stop_id in stops:
            ET.SubElement(route_element, 'link', {'refId': f"pt_{stop_id}"})

        # Create the <departures> section
        departures = ET.SubElement(transit_route, "departures")
        for idx, departure_time in enumerate(departure_times):
            ET.SubElement(
                departures, "departure", 
                id=str(idx + 1),  
                departureTime=departure_time, 
                vehicleRefId=f"pt_{route}{route_suffix}_vehicle_{idx + 1}"
            )

# Write the XML tree to a file
tree = ET.ElementTree(transit_schedule)
tree.write(r"C:\Users\zhous\Desktop\zhouyh\berlin project\bus\bus_schedule.xml")